<h2 style='color:#09a2b3'> Batch Inference</h2>

In questa demo faccio vedere come è possibile create un **Job per analisi immagini e fare Object Detection su N immagini**, con una sola API call

(molto veloce)

In [1]:
import oci
import torch

from oci.ai_vision import AIServiceVisionClient
from oci.ai_vision.models import AnalyzeImageDetails
from oci.ai_vision.models import ImageObjectDetectionFeature
from oci.ai_vision.models import InlineImageDetails
from oci.ai_vision.models import ObjectStorageImageDetails, ObjectLocation, OutputLocation

import os
import numpy as np
import pandas as pd
import glob

In [2]:
# get the RP signer
rps = oci.auth.signers.get_resource_principals_signer()

In [3]:
COMPARTMENT_ID="ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a"

ENDPOINT = "https://vision.aiservice.eu-frankfurt-1.oci.oraclecloud.com"
# the custom trained model we're using for Object Detection
MODEL_ID = "ocid1.aivisionmodel.oc1.eu-frankfurt-1.amaaaaaangencdyarrrcfnwo2yly3goldt4zwvvcs2bkerzwfrbntli74sta"

ai_service_vision_client = AIServiceVisionClient(config={}, service_endpoint=ENDPOINT, signer=rps)

In [9]:
%%time

# questi sono file nel bucket... non in locale
fname_list = ['oracle1.jpeg', 'oracle2.jpeg', 'oracle_red_bull1.jpeg', 'oracle_red_bull2.jpeg']

my_object_locations = [ObjectLocation(namespace_name="frqap2zhtzbe",
                                      bucket_name="batch_image_input", object_name=x) for x in fname_list]

# Send the request to service, some parameters are not required, see API
# doc for more info
create_image_job_response = ai_service_vision_client.create_image_job(
    create_image_job_details=oci.ai_vision.models.CreateImageJobDetails(
        input_location=oci.ai_vision.models.ObjectListInlineInputLocation(
            source_type="OBJECT_LIST_INLINE_INPUT_LOCATION",
            # here I can handle multiple files
            # costruisco dinamicamente
            object_locations=my_object_locations),
        features=[
            ImageObjectDetectionFeature(
                # qui forniamo il custom model id
                model_id=MODEL_ID)],
        output_location=OutputLocation(
            namespace_name="frqap2zhtzbe",
            bucket_name="batch_image_output",
            prefix="job2"),
        compartment_id=COMPARTMENT_ID,
        display_name="ImageJob1",
        is_zip_output_enabled=True))

# Get the data from response
print(create_image_job_response.data)

{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a",
  "display_name": "ImageJob1",
  "features": [
    {
      "feature_type": "OBJECT_DETECTION",
      "max_results": null,
      "model_id": "ocid1.aivisionmodel.oc1.eu-frankfurt-1.amaaaaaangencdyarrrcfnwo2yly3goldt4zwvvcs2bkerzwfrbntli74sta"
    }
  ],
  "id": "ocid1.aivisionimagejob.oc1.eu-frankfurt-1.amaaaaaa74akfsaa3qzmwpfas4gu577fnq2hn6r7at2vfcgukzlmfrpzoyna",
  "input_location": {
    "object_locations": [
      {
        "bucket_name": "batch_image_input",
        "namespace_name": "frqap2zhtzbe",
        "object_name": "oracle2.jpeg"
      },
      {
        "bucket_name": "batch_image_input",
        "namespace_name": "frqap2zhtzbe",
        "object_name": "oracle_red_bull1.jpeg"
      },
      {
        "bucket_name": "batch_image_input",
        "namespace_name": "frqap2zhtzbe",
        "object_name": "oracle1.jpeg"
      },
      {
        "bucket_name": "batch_image_

In [10]:
JOB_ID = create_image_job_response.data.id

JOB_ID

'ocid1.aivisionimagejob.oc1.eu-frankfurt-1.amaaaaaa74akfsaa3qzmwpfas4gu577fnq2hn6r7at2vfcgukzlmfrpzoyna'

In [12]:
#
# per verificare se il JOB è terminato
#
response = ai_service_vision_client.get_image_job(image_job_id=JOB_ID)

response.data

{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaahioj3gxhxpebugaocrtfbb25yramvyfs4ah2prmhnf2adovtmp6a",
  "display_name": "ImageJob1",
  "features": [
    {
      "feature_type": "OBJECT_DETECTION",
      "max_results": null,
      "model_id": "ocid1.aivisionmodel.oc1.eu-frankfurt-1.amaaaaaangencdyarrrcfnwo2yly3goldt4zwvvcs2bkerzwfrbntli74sta"
    }
  ],
  "id": "ocid1.aivisionimagejob.oc1.eu-frankfurt-1.amaaaaaa74akfsaa3qzmwpfas4gu577fnq2hn6r7at2vfcgukzlmfrpzoyna",
  "input_location": {
    "object_locations": [
      {
        "bucket_name": "batch_image_input",
        "namespace_name": "frqap2zhtzbe",
        "object_name": "oracle2.jpeg"
      },
      {
        "bucket_name": "batch_image_input",
        "namespace_name": "frqap2zhtzbe",
        "object_name": "oracle_red_bull1.jpeg"
      },
      {
        "bucket_name": "batch_image_input",
        "namespace_name": "frqap2zhtzbe",
        "object_name": "oracle1.jpeg"
      },
      {
        "bucket_name": "batch_image_